<a href="https://colab.research.google.com/github/Nadian-Ali/Temporary_Vid_classification/blob/main/sampleVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Projects/Video_Rec/'

/content/drive/My Drive/Projects/Video_Rec


In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
%ls

Dataset/  train_data.npy  train_mask.npy  val_data.npy  val_mask.npy  video.avi


In [ ]:
train_data = np.load('train_data.npy')
val_data = np.load('val_data.npy')

In [ ]:
print(np.shape(train_data))
print(np.shape(val_data))


(13, 500, 1024)
(6, 500, 1024)


In [ ]:
root =  '/content/drive/My Drive/Projects/Video_Rec/'
data_set      = 'Dataset'

train         = 'train'
val           = 'val'
train_ann     = 'train_action_classes.csv'
val_ann       = 'val_action_classes.csv'
videos_pth    = os.path.join(root,data_set)
train_df_path = os.path.join(root,data_set,train_ann)
val_df_path   = os.path.join(root,data_set,val_ann)
path_to_train = '/content/drive/My Drive/Projects/Video_Rec/Dataset/train'
path_to_val   = '/content/drive/My Drive/Projects/Video_Rec/Dataset/val'


##
Num_Segs = 20
max_frames= 500
feature_size = 1024

#Assumin actions are centered in the video :
<ul>
  <li> Action starts a few frames after start</li>
  <li> Actions finish by the end of the videio</li>
  <li> Actions is well distributed with in the video</li>

lets sample the video

If video has T frames; 
* we will select 20 segements from it ! 
* for started we take a representative frame from each segement 
* so take the video 
* dive into 20 segements
* take the middle frame 



#load the csv files again 
train_df = pd.read_csv(train_df_path)

In [ ]:
train_df = pd.read_csv(train_df_path)
val_df =  pd.read_csv(val_df_path)

In [ ]:
# val_df.head(10)

# Extract class labels.

In [ ]:
train_df['label'] = train_df['action_class'].map(lambda x: 1 if 'take' in x else 0)
train_labels = np.array(train_df['label'].tolist())
print(train_labels)

[1 1 1 0 1 0 1 0 1 1 1 0 0]


In [ ]:
val_df['label'] = val_df['action_class'].map(lambda x: 1 if 'take' in x else 0)
val_labels = np.array(val_df['label'].tolist())
print(val_labels)

[0 1 1 1 0 0]


<h3>Get number of frames in the video</h3>

In [ ]:
def num_frames(df, start_rame = 'start_frame', stop_frame = 'stop_frame'):

  return (df['stop_frame']-df['start_frame']).tolist() 

print('num training frames is: ',num_frames(train_df))
print('num training frames is: ',num_frames(val_df))
print('num training frames is: ',num_frames(val_df.loc[0]))

num training frames is:  [54, 112, 109, 111, 86, 630, 57, 47, 142, 131, 90, 95, 150]
num training frames is:  [461, 69, 102, 197, 807, 89]
num training frames is:  461


<h3>Get index of frames that are going to be fed into the network </h3>


In [ ]:
def resample_frames(frame,Num_Seg=Num_Segs):
    
  interval = frame//Num_Seg
#pad or truncate frames / padd with last frame 
  if (frame%Num_Seg) > (Num_Seg/2):
        #3                10
    to_pad =  (Num_Seg - frame%Num_Seg)
    B = np.pad(np.array(list(range(frame))),(0,to_pad),constant_values = frame)

  elif (frame%Num_Seg) <= (Num_Seg/2):
    to_remove =  frame%Num_Seg
    B = list(range(frame-to_remove))
#  create chuncks    
  interval = len(B)//Num_Seg
  w = B[::interval]

#sample middle frames 
  middle = [round((w[i+1]+w[i])/2) for i in range(len(w)-1)]
  middle.append(frame)

  return middle

# print(w,'len wanted frames = ',len(w)) 
# print(middle,'len wanted frames = ',len(middle))

In [ ]:
def get_resampled_dataset(dataset,df,Num_Segs=20):
  resampled_dataset = np.zeros((len(dataset),Num_Segs,feature_size))

  for i,feature_sequence in enumerate(dataset):
    # resampled_dataset = np.zeros()
    frames = num_frames(df.loc[i])
    frames = min(frames,max_frames-1)
    wanted_frames = resample_frames(frames,Num_Segs)
    # print(wanted_frames, 'len wanted frames is {}'.format(len(wanted_frames)))
    resampled_dataset[i,:,:] = dataset[i,wanted_frames,:]
  return resampled_dataset



<h1>create resmapled dataset</h1>

In [ ]:
resampled_train_dataset = get_resampled_dataset(dataset = train_data,df= train_df)
resampled_val_dataset =  get_resampled_dataset(dataset = val_data,df= val_df)

In [ ]:
%pwd

'/content/drive/MyDrive/Projects/Video_Rec'

<h1>Save training validation Label data</h1> 

<h4>save samples</h4>


In [ ]:
np.save('resampled_train_dataset.npy',resampled_train_dataset)
np.save('resampled_val_dataset.npy',resampled_val_dataset)


<h4>save labels</h1>

In [ ]:
np.save('train_labels.npy',train_labels)
np.save('val_labels.npy',val_labels)

<

<h1>Ignore code below its jut for my own tests</h1>

In [ ]:
%pwd

'/content/drive/MyDrive/Projects/Video_Rec'

In [ ]:
resampled_train_dataset[1,1:3,:10]

array([[8.03594303e-04, 6.56855851e-03, 4.35690861e-03, 4.15162975e-03,
        2.11536229e-01, 1.48617578e+00, 6.97014795e-04, 5.03662927e-03,
        2.93266475e-01, 8.64731672e-04],
       [9.54095274e-04, 6.79056859e-03, 5.32030361e-03, 4.98799421e-03,
        1.93116426e-01, 1.83430433e+00, 1.07741205e-03, 4.86542471e-03,
        2.74755239e-01, 1.15195953e-03]])

In [ ]:
train_data_resampled = np.zeros((len(train_data),Num_Segs,feature_size))

for i,feature_sequence in enumerate(train_data):
  frames = num_frames(train_df.loc[i])
  frames = min(frames,499)
  wanted_frames = resample_frames(frames,Num_Segs)
  print(wanted_frames, 'len wanted frames is {}'.format(len(wanted_frames)))
  train_data_resampled[i,:,:] = train_data[i,wanted_frames,:]

In [ ]:
# np.shape(train_data)


(13, 20, 1024)

In [ ]:
train_data_resampled[10,1:2,:]

array([[4.35300841e-04, 4.18961095e-03, 2.32251780e-03, ...,
        3.28398347e+00, 1.84945926e-01, 1.27788126e+00]])

In [ ]:

def get_idx_samples(df,num_segs=Num_Segs):


  frames = num_frames(df)
  m = 
  frames = min(max_frames,frames)
  intervals = round(frames/num_segs)
#  return list(range(df['start_frame'],min(df['stop_frame'],max_frames),intervals)),frames
  # wanted_frames  = [list(range(0,frames,intervals))]
  indices= list(range(0,frames+1,intervals)) 
  if len(indices)<Num_Segs:
    indices.append(frames)
  
  # print(type(indices))
  # print(frames,'this is frmaes')
  indices.append(frames)
  wanted_frames = [round((indices[i]+indices[i+1])/2) for i in range(len(indices)-1)]
  
  return intervals,indices,wanted_frames,frames
  
intervals,indices,wanted_frames,frames = get_idx_samples(train_df.loc[2],Num_Segs)

print('intervals: ',intervals)
# print('len intervals =',len(intervals))
print('len frames is {}'.format(frames))
print(indices)
print(wanted_frames)
print('len frames =',len(wanted_frames))

intervals:  5
len frames is 109
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 109]
[2, 8, 12, 18, 22, 28, 32, 38, 42, 48, 52, 58, 62, 68, 72, 78, 82, 88, 92, 98, 102, 107]
len frames = 22


# Get needed frames 

In [ ]:
train_samples = np.shape(train_data)[0]
val_samples = np.shape(val_data)[0]

print('number of training samples is {}'.format(train_samples))
print('number of trainvalidation  samples is {}'.format(val_samples))

print('labels for class take in train data is {}'.format(len(train_labels)))
print('labels for class open in validation data is {}'.format(len(val_labels)))

number of training samples is 13
number of trainvalidation  samples is 6
labels for class take in train data is 13
labels for class open in validation data is 6


In [ ]:

# do for training data
df = train_df

new_train = np.zeros((train_samples,Num_Segs,feature_size))
# fed

for i in range(4,5):
    # wanted_frame_in_a_vedio = train_df.loc[i]
    wanted_frames ,frames = get_idx_samples(df.loc[i],num_segs=Num_Segs)
    print(i)
    print(wanted_frames)
    print(feature_size)
    new_train = train_data[i,wanted_frames,:]



#do for validation data






4
[2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70, 74, 78, 82]
1024


In [ ]:
# a = list(range(0,21))
# m= len(a) % 20
# Num_Seg = 20
# if m >(2%Num_Seg)

frame = 52

Num_Seg= 20
extr = 43%20
interval = frame//Num_Seg
if (frame%Num_Seg) > (Num_Seg/2):
      #3                10
  frame = frame + (Num_Seg - frame%Num_Seg)
elif (frame%Num_Seg) <= (Num_Seg/2):
  frame = frame - frame%Num_Seg
  
print(interval)
print(frame)

2
60


In [ ]:
frame = 1524
Num_Seg = 20
interval = frame//Num_Seg

if (frame%Num_Seg) > (Num_Seg/2):
      #3                10
  to_pad =  (Num_Seg - frame%Num_Seg)
  B = np.pad(np.array(list(range(frame))),(0,to_pad),constant_values = frame)

elif (frame%Num_Seg) <= (Num_Seg/2):
  to_remove =  frame%Num_Seg
  B = list(range(frame-to_remove))
  


interval = len(B)//Num_Seg
w = B[::interval]

middle = [round((w[i+1]+w[i])/2) for i in range(len(w)-1)]
middle.append(frame)

print(w,'len wanted frames = ',len(w)) 
print(middle,'len wanted frames = ',len(middle))

[0, 76, 152, 228, 304, 380, 456, 532, 608, 684, 760, 836, 912, 988, 1064, 1140, 1216, 1292, 1368, 1444] len wanted frames =  20
[38, 114, 190, 266, 342, 418, 494, 570, 646, 722, 798, 874, 950, 1026, 1102, 1178, 1254, 1330, 1406, 1524] len wanted frames =  20


In [ ]:
def resample_video_features(frame,Num_Seg):
    
  interval = frame//Num_Seg
#pad or truncate frames / padd with last frame 
  if (frame%Num_Seg) > (Num_Seg/2):
        #3                10
    to_pad =  (Num_Seg - frame%Num_Seg)
    B = np.pad(np.array(list(range(frame))),(0,to_pad),constant_values = frame)

  elif (frame%Num_Seg) <= (Num_Seg/2):
    to_remove =  frame%Num_Seg
    B = list(range(frame-to_remove))
#  create chuncks    
  interval = len(B)//Num_Seg
  w = B[::interval]

#sample middle frames 
  middle = [round((w[i+1]+w[i])/2) for i in range(len(w)-1)]
  # middle.append(frame)

  return middle

# print(w,'len wanted frames = ',len(w)) 
# print(middle,'len wanted frames = ',len(middle))

19
39
40
